In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df = pd.read_csv('data/Top_Influencers.csv', encoding_errors='ignore')
df

,Rank,Channel Info,Influence Score,Followers,Avg. Likes,Posts,60-Day Eng Rate,New Post Avg. Likes,Total Likes,Country Or Region
0,1,\ncristiano,92,485.2m,8.7m,3.4k,0.013,6.3m,29.1b,Spain
1,2,kyliejenner,91,370.7m,8.2m,7.0k,0.014,5.0m,57.4b,United States
2,3,\nleomessi,90,363.9m,6.7m,915,0.010,3.5m,6.1b,NaN
3,4,\nselenagomez,93,348.8m,6.1m,1.9k,0.005,1.7m,11.4b,United States
4,5,\ntherock,91,339.4m,1.8m,6.8k,0.003,932.0k,12.6b,United States
...,...,...,...,...,...,...,...,...,...,...
195,196,\nkhabib_nurmagomedov,63,33.6m,680.6k,4.6k,0.009,305.7k,3.1b,Russia
196,197,\ndanbilzerian,84,33.6m,2.2m,1.4k,0.064,2.1m,3.0b,Canada
197,198,\nraisa6690,80,33.6m,227.8k,4.2k,0.003,103.2k,955.9m,Indonesia
198,199,\nluansantana,79,33.5m,193.3k,865,0.003,82.6k,167.2m,Brazil


In [3]:
# need to preprocess some columns - convert to integers
cols_for_num = ['Followers', 'Avg. Likes', 'Posts', 'New Post Avg. Likes', 'Total Likes']

for col in cols_for_num:
    for i in range(len(df[col])):
       if str(df[col][i])[-1:]=='k':
            df[col][i] = str(df[col][i])[0:len(str(df[col][i]))-3]+ str(df[col][i])[-2:-1]+'00'
       elif str(df[col][i])[-1:]=='m':
            df[col][i] = str(df[col][i])[0:len(str(df[col][i]))-3]+ str(df[col][i])[-2:-1]+'00000'
       elif str(df[col][i])[-1:]=='b':
            df[col][i] = str(df[col][i])[0:len(str(df[col][i]))-3]+ str(df[col][i])[-2:-1]+'00000000'
       else:
            df[col][i] = str(df[col][i])

for col in cols_for_num:
     df[col] = pd.to_numeric(df[col], errors='coerce')

df.sample(5)

,Rank,Channel Info,Influence Score,Followers,Avg. Likes,Posts,60-Day Eng Rate,New Post Avg. Likes,Total Likes,Country Or Region
13,14,\ntaylorswift,91,226100000,2400000,542,0.015,3300000,1300000000,United States
34,35,\ndualipa,74,87300000,2100000,1300,0.017,1500000,2700000000,United Kingdom
145,146,\ncznburak,80,41400000,837900,1800,0.011,411100,1500000000,NaN
5,6,\narianagrande,92,333000000,3700000,5000,0.009,3100000,18500000000,United States
31,32,\nfcbarcelona,90,112500000,1100000,10000,0.007,807200,11500000000,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rank                 200 non-null    int64  
 1   Channel Info         200 non-null    object 
 2   Influence Score      200 non-null    int64  
 3   Followers            200 non-null    int64  
 4   Avg. Likes           200 non-null    int64  
 5   Posts                200 non-null    int64  
 6   60-Day Eng Rate      200 non-null    float64
 7   New Post Avg. Likes  200 non-null    int64  
 8   Total Likes          200 non-null    int64  
 9   Country Or Region    137 non-null    object 
dtypes: float64(1), int64(7), object(2)
memory usage: 15.8+ KB


In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df.describe()

,Rank,Influence Score,Followers,Avg. Likes,Posts,60-Day Eng Rate,New Post Avg. Likes,Total Likes
count,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000,200.00000
mean,100.50000,81.85000,78491000.00000,1785050.00000,3460.42500,0.01916,1224083.00000,3644949000.00000
std,57.87918,8.87532,74920154.61032,2192133.48407,3327.88220,0.03258,1857624.20001,5563069303.62039
min,1.00000,22.00000,33400000.00000,63500.00000,20.00000,0.00000,0.00000,22800000.00000
25%,50.75000,80.00000,40675000.00000,499050.00000,938.25000,0.00400,218775.00000,1075000000.00000
50%,100.50000,84.00000,50950000.00000,1100000.00000,2100.00000,0.00950,567150.00000,2050000000.00000
75%,150.25000,86.00000,70500000.00000,2125000.00000,5025.00000,0.01725,1300000.00000,4025000000.00000
max,200.00000,93.00000,485200000.00000,15400000.00000,16900.00000,0.25000,12900000.00000,57400000000.00000
